In [1]:
import csv
import requests
import time

def fetch_products(url):
    # Fetch data from Rainforest API
    params = {
        'api_key': 'A574482E8F3B4450A249A3D1011536B1',
        'type': 'bestsellers',
        'url': url
    }
    api_result = requests.get('https://api.rainforestapi.com/request', params)
    response_json = api_result.json()
    products = response_json.get('bestsellers', [])

    updated_products = []
    for product in products:
        # Extract ASIN from the product data
        asin = product.get('asin')

        # Fetch category information using the ASIN
        category_info = fetch_category_info_with_retry(asin)

        # Update the product data with category information
        product['category_id'] = category_info.get('category_id', '')
        product['category_name'] = category_info.get('category_name', '')

        # Fetch reviews for the current product
        reviews = fetch_reviews(asin)
        
        # Check if reviews fetching was successful
        if reviews is not None:
            print("These are the reviews for:", reviews)
            
            # Prepare the reviews string
            review_text = ""
            review_ids = []

            for review in reviews:
                if 'body' in review:
                    review_text += f"Rating: {review['rating']}, Review: {review['body']}\n"
                    review_ids.append(review['id'])
                    
                else:
                    # If 'body' key does not exist, provide a placeholder
                    review_text += f"Rating: {review.get('rating', '')}, Review: [No review text available]\n"
                    review_ids.append(review.get('id', ''))      

            # Join review IDs into a comma-separated string
            review_ids_str = ', '.join(review_ids)

            # Add reviews and review IDs to the product data
            product['reviews'] = review_text
            product['review_ids'] = review_ids_str
        else:
            # If reviews fetching failed, set empty values for reviews and review IDs
            product['reviews'] = ''
            product['review_ids'] = ''

        #updated_products.append(product)
        write_product_to_csv(file, product)

    return updated_products



def fetch_category_info_with_retry(asin, max_retries=3, backoff_factor=2):
    retries = 0
    while retries < max_retries:
        try:
            print("Attempting to fetch category info for :", asin)
    
            params = {
                'api_key': 'A574482E8F3B4450A249A3D1011536B1',
                'amazon_domain': 'amazon.com',
                'asin': asin,
                'type': 'product',
            }
    
            response = requests.get('https://api.rainforestapi.com/request', params=params) 
            product_info = response.json().get("product", {})
            category_info = product_info.get("categories", [{}])[-1]  # Last category    
            category_id = category_info.get("category_id", "")
            category_name = category_info.get("name", "")

            # If fetching category info fails, use categories_flat
            if not category_name:
                category_info_flat = product_info.get("categories_flat", "")
                category_name = category_info_flat.split(" > ")[-1]
                category_id = ""

            return {'category_id': category_id, 'category_name': category_name}
        
        except (requests.exceptions.RequestException, ValueError) as e:
            print("Error fetching category info:", e)
            retries += 1
            if retries < max_retries:
                delay = backoff_factor ** retries
                print(f"Retrying in {delay} seconds...")
                time.sleep(delay)
            else:
                print("Max retries exceeded. Skipping this product.")
                return {'category_id': '', 'category_name': ''}

def fetch_reviews(asin):
    print("Attempting to fetch reviews for :", asin)
    
    params = {
        'api_key': 'A574482E8F3B4450A249A3D1011536B1',
        'type': 'reviews',
        'amazon_domain': 'amazon.com',
        'asin': asin,
        'review_stars': 'all_stars',
        'reviewer_type': 'verified_purchase',
        'sort_by': 'most_recent',
        'page': '1',
        'max_page': '3'
    }
    try:
        response = requests.get('https://api.rainforestapi.com/request', params=params)
        response.raise_for_status()  # Raise an exception for HTTP errors
        reviews_data = response.json().get("reviews", [])
        return reviews_data
    except requests.exceptions.RequestException as e:
        print("Error fetching reviews:", e)
        return None
    except ValueError as ve:
        print("Error decoding JSON:", ve)
        return None

def write_product_to_csv(file, product):
    writer = csv.DictWriter(file, fieldnames=[
        'Product Name', 'Product ID', 'Link', 'Rating', 'Total Reviews', 
        'Price', 'Category Name', 'Category ID', 'Reviews', 'ReviewID'
    ])

    #for product in products:
        # Fetch category information using the ASIN
    category_info = fetch_category_info_with_retry(product['asin'])

    # Update the product data with category information
    product['category_id'] = category_info.get('category_id', '')
    product['category_name'] = category_info.get('category_name', '')
        
    writer.writerow({
        'Product Name': f'{product.get("title", "")}',
        'Product ID': product.get('asin', ''),
        'Link': product.get('link', ''),
        'Rating': product.get('rating', ''),
        'Total Reviews': product.get('ratings_total', ''),
        'Price': product.get('price', {}).get('raw', ''),
        'Category Name': product.get('category_name', ''),
        'Category ID': product.get('category_id', ''),
        'Reviews': product.get('reviews', ''),
        'ReviewID': product.get('review_ids', '')
    })

# Open CSV file in append mode
with open('Amazon Products Data.csv', mode='a', newline='', encoding='utf-8') as file:
    # Initialize the last row index
    last_row_index = file.tell()

    # Write header row if the file is empty
    if last_row_index == 0:
        writer = csv.DictWriter(file, fieldnames=[
            'Product Name', 'Product ID', 'Link', 'Rating', 'Total Reviews', 
            'Price', 'Category Name', 'Category ID', 'Reviews', 'ReviewID'
        ])
        writer.writeheader()

    # Iterate over each URL in the library
    url_library = [
        #'https://www.amazon.com/Best-Sellers-Computers-Accessories/zgbs/pc/ref=zg_bs_unv_pc_1_516866_2',
        #'https://www.amazon.com/Best-Sellers-Computers-Accessories/zgbs/pc/ref=zg_bs_pg_2_pc?_encoding=UTF8&pg=2',
        #'https://www.amazon.com/Best-Sellers-Home-Kitchen/zgbs/home-garden/ref=zg_bs_pg_1_home-garden?_encoding=UTF8&pg=1',
        #'https://www.amazon.com/Best-Sellers-Home-Kitchen/zgbs/home-garden/ref=zg_bs_pg_2_home-garden?_encoding=UTF8&pg=2',
        #'https://www.amazon.com/Best-Sellers-Office-Products/zgbs/office-products/ref=zg_bs_nav_office-products_0',
        'https://www.amazon.com/Best-Sellers-Office-Products/zgbs/office-products/ref=zg_bs_pg_2_office-products?_encoding=UTF8&pg=2'
        # Add more URLs here
    ]

    for url in url_library:
        fetch_products(url)
        #write_to_csv(file, products)

print("CSV file 'Amazon Products Data.csv' has been updated successfully!")

Attempting to fetch category info for : B08HV3MPFD
Attempting to fetch reviews for : B08HV3MPFD
These are the reviews for: [{'id': 'RXPPFIYIT5BU5', 'title': 'Super handy', 'body': 'I love this label maker. The labels are small but it is super neat to connect to your phone and print labels.', 'asin': 'B08HV3MPFD', 'body_html': '<span>I love this label maker. The labels are small but it is super neat to connect to your phone and print labels.</span>', 'link': 'https://www.amazon.com/gp/customer-reviews/RXPPFIYIT5BU5/ref=cm_cr_arp_d_rvw_ttl?ie=UTF8&ASIN=B08HV3MPFD', 'rating': 4, 'date': {'raw': 'Reviewed in the United States on May 10, 2024', 'utc': '2024-05-10T00:00:00.000Z'}, 'profile': {'name': 'Kasey Davis', 'link': 'https://www.amazon.com/gp/profile/amzn1.account.AFAGT6S4I7LF32JIITM43P434V2Q/ref=cm_cr_arp_d_gw_btm?ie=UTF8', 'id': 'AFAGT6S4I7LF32JIITM43P434V2Q'}, 'vine_program': False, 'verified_purchase': True, 'attributes': [{'name': 'Color', 'value': 'Green'}], 'attributes_flat': '

Attempting to fetch category info for : B0BRTCD8NK
Attempting to fetch reviews for : B0BRTCD8NK
Error fetching reviews: 500 Server Error: Internal Server Error for url: https://api.rainforestapi.com/request?api_key=A574482E8F3B4450A249A3D1011536B1&type=reviews&amazon_domain=amazon.com&asin=B0BRTCD8NK&review_stars=all_stars&reviewer_type=verified_purchase&sort_by=most_recent&page=1&max_page=3
Attempting to fetch category info for : B0BRTCD8NK
Attempting to fetch category info for : B01BYKUI9C
Attempting to fetch reviews for : B01BYKUI9C
These are the reviews for: [{'id': 'R39603P31ONQ6F', 'title': 'Perfect', 'body': "Thought I had a backup ink cartridge but turns out I didn't. I ordered this. Got this quickly and it saved the day.", 'asin': 'B01BYKUI9C', 'body_html': "<span>Thought I had a backup ink cartridge but turns out I didn't. I ordered this. Got this quickly and it saved the day.</span>", 'link': 'https://www.amazon.com/gp/customer-reviews/R39603P31ONQ6F/ref=cm_cr_arp_d_rvw_ttl?

Attempting to fetch category info for : B0CVVD46DT
Attempting to fetch reviews for : B0CVVD46DT
These are the reviews for: []
Attempting to fetch category info for : B0CVVD46DT
Attempting to fetch category info for : B082PMRSBK
Attempting to fetch reviews for : B082PMRSBK
These are the reviews for: [{'id': 'R1C5QUFXUYFNQX', 'title': 'Love them', 'body': 'I love these pens. No issues with smears. Use them all day long.', 'asin': 'B082PN4X5J', 'body_html': '<span>I love these pens. No issues with smears. Use them all day long.</span>', 'link': 'https://www.amazon.com/gp/customer-reviews/R1C5QUFXUYFNQX/ref=cm_cr_arp_d_rvw_ttl?ie=UTF8&ASIN=B082PMRSBK', 'rating': 5, 'date': {'raw': 'Reviewed in the United States on May 10, 2024', 'utc': '2024-05-10T00:00:00.000Z'}, 'profile': {'name': 'This game is awesome', 'link': 'https://www.amazon.com/gp/profile/amzn1.account.AHLNSDGEMKQ6NBHC3SMRXJKFOTDA/ref=cm_cr_arp_d_gw_btm?ie=UTF8', 'id': 'AHLNSDGEMKQ6NBHC3SMRXJKFOTDA'}, 'vine_program': False, 'ver

Attempting to fetch category info for : B0BRTGTH8R
Attempting to fetch reviews for : B0BRTGTH8R
These are the reviews for: [{'id': 'R300DGEZH9LIGX', 'title': 'Why send a card when you can send a bouquet?', 'body': 'These paper bouquets last longer, require less attention, cost less, you could custom scent them if you want to, and they are vibrant! Get with the program!', 'asin': 'B0BRTGTH8R', 'body_html': '<span>These paper bouquets last longer, require less attention, cost less, you could custom scent them if you want to, and they are vibrant! Get with the program!</span>', 'link': 'https://www.amazon.com/gp/customer-reviews/R300DGEZH9LIGX/ref=cm_cr_arp_d_rvw_ttl?ie=UTF8&ASIN=B0BRTGTH8R', 'rating': 5, 'date': {'raw': 'Reviewed in the United States on May 10, 2024', 'utc': '2024-05-10T00:00:00.000Z'}, 'profile': {'name': 'Mrs. Clark', 'link': 'https://www.amazon.com/gp/profile/amzn1.account.AHCCCGBTX2A4AWQXIFETQEN5VW5A/ref=cm_cr_arp_d_gw_btm?ie=UTF8', 'id': 'AHCCCGBTX2A4AWQXIFETQEN5VW5

Attempting to fetch category info for : B00006IFI3
Attempting to fetch reviews for : B00006IFI3
These are the reviews for: [{'id': 'R28QO09FENQM4A', 'title': 'IT DEFINITELY WORKS', 'body': 'It does not smear. I use it to write checks', 'asin': 'B00006IFI3', 'body_html': '<span>It does not smear. I use it to write checks</span>', 'link': 'https://www.amazon.com/gp/customer-reviews/R28QO09FENQM4A?ASIN=B00006IFI3', 'rating': 5, 'date': {'raw': 'Reviewed in the United States on May 9, 2024', 'utc': '2024-05-09T00:00:00.000Z'}, 'profile': {'name': 'Debbie', 'link': 'https://www.amazon.com/gp/profile/amzn1.account.AFJ7LKSCCRFRPKPMJN7IFNASVN2Q', 'id': 'AFJ7LKSCCRFRPKPMJN7IFNASVN2Q'}, 'vine_program': False, 'verified_purchase': True, 'attributes': [{'name': 'Color', 'value': 'Black'}, {'name': 'Pattern Name', 'value': 'Markers'}], 'attributes_flat': 'Color: Black, Pattern Name: Markers', 'review_country': 'us', 'is_global_review': False, 'position': 1, 'page': 1}, {'id': 'RJ3KSYVS4SQ37', 'titl

...

Attempting to fetch category info for : B0010JEJPC
Attempting to fetch reviews for : B0010JEJPC
Error fetching reviews: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Attempting to fetch category info for : B0010JEJPC
Error fetching category info: HTTPSConnectionPool(host='api.rainforestapi.com', port=443): Max retries exceeded with url: /request?api_key=A574482E8F3B4450A249A3D1011536B1&amazon_domain=amazon.com&asin=B0010JEJPC&type=product (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001B6F0D91F50>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Retrying in 2 seconds...
Attempting to fetch category info for : B0010JEJPC
Attempting to fetch category info for : B00WR23VRI
Attempting to fetch reviews for : B00WR23VRI
These are the reviews for: [{'id': 'R3LZX81DF3NCYB', 'title': 'Replacement Copier ink', 'body': 'Good quality copier ink', 'asin': 'B00WR23VRI', 'body_html': '<span>G

Attempting to fetch category info for : B0BG53HYHW
Attempting to fetch reviews for : B0BG53HYHW
These are the reviews for: [{'id': 'R28CGOLQTY848O', 'title': 'Good quality!', 'body': 'These do not go onto the next page. They are also the exact color as shown in the photos.', 'asin': 'B0BG53HYHW', 'body_html': '<span>These do not go onto the next page. They are also the exact color as shown in the photos.</span>', 'link': 'https://www.amazon.com/gp/customer-reviews/R28CGOLQTY848O/ref=cm_cr_arp_d_rvw_ttl?ie=UTF8&ASIN=B0BG53HYHW', 'rating': 4, 'date': {'raw': 'Reviewed in the United States on May 10, 2024', 'utc': '2024-05-10T00:00:00.000Z'}, 'profile': {'name': 'They were not true to size. Very BIG!!', 'link': 'https://www.amazon.com/gp/profile/amzn1.account.AFGFSQ6VL64JP7LX3XL2J2NFLIGA/ref=cm_cr_arp_d_gw_btm?ie=UTF8', 'id': 'AFGFSQ6VL64JP7LX3XL2J2NFLIGA'}, 'vine_program': False, 'verified_purchase': True, 'attributes': [{'name': 'Color', 'value': 'Candy Colors'}, {'name': 'Size', 'value

Attempting to fetch category info for : B0CRGCV8JJ
Attempting to fetch reviews for : B0CRGCV8JJ
These are the reviews for: [{'id': 'R21WRQV10PJN3Q', 'title': 'No good', 'body': 'These pens are horrible. They had the right with the tips break and wear out very very fast.', 'asin': 'B08BX66VLD', 'body_html': '<span>These pens are horrible. They had the right with the tips break and wear out very very fast.</span>', 'link': 'https://www.amazon.com/gp/customer-reviews/R21WRQV10PJN3Q?ASIN=B0CRGCV8JJ', 'rating': 1, 'date': {'raw': 'Reviewed in the United States on May 9, 2024', 'utc': '2024-05-09T00:00:00.000Z'}, 'profile': {'name': 'steve lesser', 'link': 'https://www.amazon.com/gp/profile/amzn1.account.AFUVBZMRRIFFXFADWCOC5E6BLHXA', 'id': 'AFUVBZMRRIFFXFADWCOC5E6BLHXA'}, 'vine_program': False, 'verified_purchase': True, 'attributes': [{'name': 'Color', 'value': 'White'}, {'name': 'Size', 'value': '1 Count (Pack of 4)'}], 'attributes_flat': 'Color: White, Size: 1 Count (Pack of 4)', 'review

Attempting to fetch category info for : B07MCYDD62
Attempting to fetch reviews for : B07MCYDD62
These are the reviews for: [{'id': 'RPQN1A41WF8K3', 'title': 'Grandpa H', 'body': 'Good quality', 'asin': 'B07MCYDD62', 'body_html': '<span>Good quality</span>', 'link': 'https://www.amazon.com/gp/customer-reviews/RPQN1A41WF8K3/ref=cm_cr_arp_d_rvw_ttl?ie=UTF8&ASIN=B07MCYDD62', 'rating': 5, 'date': {'raw': 'Reviewed in the United States on May 8, 2024', 'utc': '2024-05-08T00:00:00.000Z'}, 'profile': {'name': 'Grandpa H', 'link': 'https://www.amazon.com/gp/profile/amzn1.account.AFBPQ57ENTNSYNICZTRQPFLXWXSQ/ref=cm_cr_arp_d_gw_btm?ie=UTF8', 'id': 'AFBPQ57ENTNSYNICZTRQPFLXWXSQ'}, 'vine_program': False, 'verified_purchase': True, 'review_country': 'us', 'is_global_review': False, 'position': 1, 'page': 1}, {'id': 'R2TIZFV6QL2NAU', 'title': 'Clear, crisp colors', 'body': 'Clear, consistent, high quality ink replacement for HP ink jets.', 'asin': 'B07MCYDD62', 'body_html': '<span>Clear, consistent, 

CSV file 'Amazon Products Data.csv' has been updated successfully!
